In [23]:
import pandas as pd
import numpy as np
import pandas as pd
from geotext import GeoText

import nest_asyncio
nest_asyncio.apply()

import twint

In [4]:
old_tweets = pd.read_csv('data/jan-july-2020.csv', date_parser=True)

In [5]:
old_tweets.head()

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,2020-07-09 17:42:45,RhinobotStudios,NaN,2,0,5,That moment when you learn about @Podchaser 's...,NaN,@Podchaser,#PodRevDay #facepalm,1281282657339441153,https://twitter.com/RhinobotStudios/status/128...
1,2020-07-09 17:30:21,_joe_stone_,NaN,0,1,1,... a little late for #podrevday but check out...,NaN,@bigswingpodcast,#podrevday,1281279539193106433,https://twitter.com/_joe_stone_/status/1281279...
2,2020-07-09 11:18:43,JTTG,NaN,0,2,5,(shared the wrong review didn't I) THIS IS MY ...,NaN,@LetsFightABoss @Podchaser,#PodRevDay,1281186011297583104,https://twitter.com/JTTG/status/12811860112975...
3,2020-07-09 10:53:51,podsnaija,dsmileoutreach,1,1,3,#AbujaTwitterCommunity #Abuja #Africa #podcast...,NaN,NaN,#AbujaTwitterCommunity #Abuja #Africa #podcast...,1281179755920982017,https://twitter.com/podsnaija/status/128117975...
4,2020-07-09 09:14:57,CastingLotsPod,PodRevDay,0,0,1,Oops! Sorry We've come at this backwards (our ...,NaN,NaN,NaN,1281154864660336645,https://twitter.com/CastingLotsPod/status/1281...


In [7]:
old_tweets['date'].max()

'2020-07-09 17:42:45'

In [8]:
!GetOldTweets3 --querysearch "podrevday" --since 2020-07-09 --until 2020-08-11 --output "data/july_august_2020.csv"

Saved 477
Done. Output file generated "data/july_august_2020.csv".


In [9]:
new_tweets = pd.read_csv('data/july_august_2020.csv', parse_dates=True)

In [10]:
new_tweets.head()

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,2020-08-10 20:32:47,PodRevDay,LizWilcox79,0,0,4,"ohhh, thats a dangerous bit of info. I am terr...",NaN,NaN,NaN,1292921860225736709,https://twitter.com/PodRevDay/status/129292186...
1,2020-07-22 16:38:32,BiobridgesH2020,NaN,0,0,3,Biobridges braucht Ihre Antworten. Nehmen Sie ...,NaN,NaN,NaN,1285977538955808768,https://twitter.com/BiobridgesH2020/status/128...
2,2020-08-10 20:07:39,LizWilcox79,PodRevDay,1,0,4,"You can mark episodes as ""listened"" on Podchas...",NaN,NaN,NaN,1292915534413979648,https://twitter.com/LizWilcox79/status/1292915...
3,2020-08-10 19:59:17,PodRevDay,podcast_junkies,0,0,0,"omg, you did it. ((((dancing))))",NaN,NaN,NaN,1292913431037857794,https://twitter.com/PodRevDay/status/129291343...
4,2020-08-10 19:58:39,PodRevDay,VincentMacElroy,1,0,0,What's this podcast tracking website? Would lo...,NaN,NaN,NaN,1292913272421855233,https://twitter.com/PodRevDay/status/129291327...


In [20]:
#main_list = np.setdiff1d(list_2,list_1)
old_users = old_tweets.username.unique()
august_users = new_tweets.username.unique()
new_users = np.setdiff1d(august_users, old_users)

In [26]:
c = twint.Config()
c.Store_object = True
c.Pandas = True

for user in new_users: 
    c.Username = user
    twint.run.Lookup(c)
Users_df = twint.storage.panda.User_df

users_df = Users_df.drop_duplicates()
users_df.to_csv('data/new_user_data.csv', index=False)

1122247436217483264 | 30&NerdyPodcast | @30AndNerdyPod | Private: 0 | Verified: 0 | Bio: 30&Nerdy Podcast is for you, the nerds! Join us as we live the nerdy life. Join us as The Nerd World Order takes over. https://linktr.ee/30AndNerdyPodcast … | Location: Atlanta, GA | Url: http://www.30andnerdypodcast.com | Joined: 27 Apr 2019 2:13 PM | Tweets: 2389 | Following: 457 | Followers: 541 | Likes: 2303 | Media: 269 | Avatar: https://pbs.twimg.com/profile_images/1260575960379908098/XT8K_g2a_400x400.jpg
1123770623199731713 | 3 Hours Later 🎙 | @3hrs_later | Private: 0 | Verified: 0 | Bio: What started as an online argument between friends is now a podcast! With your host: Alex @ultimate_doyer Matt @incredimatt Hannah @chimpmilitia #PodNation | Location: Fresno, CA | Url: http://linktr.ee/3hourslater | Joined: 1 May 2019 7:06 PM | Tweets: 10995 | Following: 3056 | Followers: 4353 | Likes: 14087 | Media: 2278 | Avatar: https://pbs.twimg.com/profile_images/1265697154191908865/bm4yQ-KA_400x400.j

In [42]:
df_new_users = pd.read_csv('data/new_user_data.csv')
df_new_users= df_new_users.loc[:, ['id','username', 'name', 'location', 'join_date', 'followers', 'following', 'likes', 'url', 'verified' ]]

In [45]:
def location_extraction (df):
    '''Creates creates a geotext column to extract city and country info if possible'''

    df.loc[:, "location"] = df.loc[:, "location"].fillna("blank")
    df.loc[:, "geotext"] = df.loc[:, "location"].apply(GeoText)
    df.loc[:, 'city'] = df.loc[:, 'geotext'].apply(lambda x: x.cities)
    df.loc[:, 'country'] = df.loc[:, 'geotext'].apply(lambda x: x.countries)
    
    return df

In [46]:
df_new_users = location_extraction(df_new_users)

In [47]:
df_new_users.head()

,id,username,name,location,join_date,followers,following,likes,url,verified,geotext,city,country
0,1122247436217483264,30AndNerdyPod,30&NerdyPodcast,"Atlanta, GA",27 Apr 2019,541,457,2303,http://www.30andnerdypodcast.com,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,[Atlanta],[]
1,1123770623199731713,3hrs_later,3 Hours Later 🎙,"Fresno, CA",1 May 2019,4353,3056,14087,http://linktr.ee/3hourslater,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,[Fresno],[]
2,820303909725929473,AnnCSloan,AnnCSloan Writes The Carlötta Beautox Chronicles,Los Angeles,14 Jan 2017,185,221,2638,NaN,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,[Los Angeles],[]
3,1067081514997956610,BiobridgesH2020,Biobridges Project,blank,26 Nov 2018,1254,263,1144,http://www.biobridges-project.eu,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,[],[]
4,1229250265775099907,Browngirlsread1,Brown Girls Read Pod,California,16 Feb 2020,679,783,3036,https://browngirlsread.wordpress.com/,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,[],[]


In [50]:
from geonamescache import GeonamesCache
gc = GeonamesCache()
countries = gc.get_countries()
country_info = pd.DataFrame(countries).T
country_info = country_info.set_index('geonameid').reset_index()
name_code = country_info.loc[:, ["name", "iso3"]]

us_states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
             "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA",
             "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY",
             "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX",
             "UT", "VT", "VA", "WA", "WV", "WI", "WY", "USA", "United States",
             'Seattle', "Los Angeles", "Houston", "Atlanta", "Pittsburgh"]

us_state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", 
               "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", 
               "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", 
               "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
               "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", 
               "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", 
               "Wisconsin", "West Virginia", "Wyoming"]

can_prov_abbrev = {'Alberta': 'AB','British Columbia': 'BC','Manitoba': 'MB', 'New Brunswick': 'NB',
                       'Newfoundland and Labrador': 'NL', 'Northwest Territories': 'NT','Nova Scotia': 'NS','Nunavut': 'NU',
                       'Ontario': 'ON','Prince Edward Island': 'PE', 'Quebec': 'QC','Saskatchewan': 'SK','Yukon': 'YT'}

can_prov_names, can_prov_abbr = zip(*can_prov_abbrev.items())

uk = ["England", 'Wales', "Scotland", 'London', "Manchester", "Isle of Wight", "Northern Ireland", "United Kingdom", 'Bailiwick of Guernsey', "UK", "Hoxton", "Jersey"]

india_city = ["Bangalore", "Delhi", "Hyderabad", "Bengaluru"]

german_city = ["Munich", "Berlin", "eisgau","Hamburg", "Dortmund"]

south_africa = ["South Africa", "Durban", "Johannesburg"]

def replacer(df, area, name):
    df.loc[(df.location.str.contains('|'.join(area))), "country"] = name    
    return df

In [51]:
df_new_users = replacer(df_new_users, can_prov_names, "Canada")
df_new_users = replacer(df_new_users, can_prov_abbrev, "Canada")
df_new_users = replacer(df_new_users, us_state_names, "United States")
df_new_users = replacer(df_new_users, us_states, "United States")
df_new_users = replacer(df_new_users, uk, "United Kingdom")
df_new_users = replacer(df_new_users, german_city, "Germany")
df_new_users = replacer(df_new_users, south_africa, "South Africa")
df_new_users = replacer(df_new_users, india_city, "India")
df_new_users.loc[(df_new_users.location == "Italia"), "country"] = "Italy" 
df_new_users.loc[(df_new_users.location == "Belgrade"), "country"] = "Serbia" 
df_new_users.loc[(df_new_users.country == "PolandSerbia"), "country"] = "Poland" 

In [59]:
df_new_users.tail(10)

,id,username,name,location,join_date,followers,following,likes,url,verified,geotext,city,country
55,1234538654057533440,lislislistenpod,Listen! Listen! Listen! With Curtis Elton,blank,2 Mar 2020,955,822,7128,https://podcasts.apple.com/us/podcast/listen-l...,0,<geotext.geotext.GeoText object at 0x7f7cc055a...,[],[]
56,2293594190,podcast_junkies,Podcast Junkies,"Minneapolis, MN",15 Jan 2014,6228,5328,6149,http://pjnk.es/subscribe,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[Minneapolis],United States
57,1280322824184397824,podcasterprice,podcasterprice,Washington DC,6 Jul 2020,25,30,51,https://pricepodcast.libsyn.com/,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[Washington],United States
58,14019652,rezzz,Jason Resnick 👨‍👩‍👦‍👦🤘,"Oceanside, NY",26 Feb 2008,3319,1135,14658,http://feastclub.co,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[Oceanside],United States
59,1249376400290844673,seanickroutepod,The SeaNick Route Podcast,blank,12 Apr 2020,42,168,126,NaN,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[],[]
60,1135571328591892481,theJaMcastpod,The JaMcast Pod 🎙🇮🇪,JaMcast Studio,3 Jun 2019,1286,1724,1817,https://open.spotify.com/episode/080Fqmt2J3kjK...,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[],[]
61,1195442463911153670,thelevelupcrew,The Level Up Crew,blank,15 Nov 2019,853,1196,4504,https://anchor.fm/leveluporshutup,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[],[]
62,1206443593289936898,truepopoholic,Brian Dupre,"Florida, USA",15 Dec 2019,20,122,1270,https://itunes.apple.com/us/podcast/popoholics...,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[Florida],United States
63,897475742707200000,vladmihajlovic,Vladimir Mihajlović,Poland,15 Aug 2017,112,311,219,NaN,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[],[Poland]
64,202990434,w_jahr,Wissenschaftsjahr,blank,15 Oct 2010,10190,1323,10122,http://www.wissenschaftsjahr.de,1,<geotext.geotext.GeoText object at 0x7f7cc83dc...,[],[]


In [60]:
def list_to_string(df):
    df["city"] = df['city'].apply(lambda x: "".join(map(str, x)))
    df["country"] = df['country'].apply(lambda x: "".join(map(str, x)))
    
    return df

In [62]:
df_new_users = list_to_string(df_new_users)

In [63]:
df_new_users

,id,username,name,location,join_date,followers,following,likes,url,verified,geotext,city,country
0,1122247436217483264,30AndNerdyPod,30&NerdyPodcast,"Atlanta, GA",27 Apr 2019,541,457,2303,http://www.30andnerdypodcast.com,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,Atlanta,United States
1,1123770623199731713,3hrs_later,3 Hours Later 🎙,"Fresno, CA",1 May 2019,4353,3056,14087,http://linktr.ee/3hourslater,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,Fresno,United States
2,820303909725929473,AnnCSloan,AnnCSloan Writes The Carlötta Beautox Chronicles,Los Angeles,14 Jan 2017,185,221,2638,NaN,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,Los Angeles,United States
3,1067081514997956610,BiobridgesH2020,Biobridges Project,blank,26 Nov 2018,1254,263,1144,http://www.biobridges-project.eu,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,,
4,1229250265775099907,Browngirlsread1,Brown Girls Read Pod,California,16 Feb 2020,679,783,3036,https://browngirlsread.wordpress.com/,0,<geotext.geotext.GeoText object at 0x7f7cc0562...,,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1135571328591892481,theJaMcastpod,The JaMcast Pod 🎙🇮🇪,JaMcast Studio,3 Jun 2019,1286,1724,1817,https://open.spotify.com/episode/080Fqmt2J3kjK...,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,,
61,1195442463911153670,thelevelupcrew,The Level Up Crew,blank,15 Nov 2019,853,1196,4504,https://anchor.fm/leveluporshutup,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,,
62,1206443593289936898,truepopoholic,Brian Dupre,"Florida, USA",15 Dec 2019,20,122,1270,https://itunes.apple.com/us/podcast/popoholics...,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,Florida,United States
63,897475742707200000,vladmihajlovic,Vladimir Mihajlović,Poland,15 Aug 2017,112,311,219,NaN,0,<geotext.geotext.GeoText object at 0x7f7cc83dc...,,Poland
